In [3]:
from pyspark.sql import *
import findspark
from pyspark.sql.types import *
findspark.init()
import statistics
import os
import re
import gc
from time import time
import datatable
import numpy as np
import pandas as pd
spark = SparkSession.builder.master("local[*]").config('spark.dynamicAllocation.enabled','true').config("spark.debug.maxToStringFields", "100").config("spark.sql.execution.arrow.enable","true").config("spark.executor.memory","20g").config("spark.driver.memory", "20g").config("spark.core.connection.ack.wait.timeout","36000s").config("spark.executor.heartbeatInterval","36000s").config("spark.network.timeout", "50000s").config("spark.rpc.lookupTimeout", "5000s").config("spark.shuffle.io.connectionTimeout", "50000s").appName("data").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [4]:
# #读取B2表
path1 = r'D:\Output\zhuhao\B1'
# file_name_list = ['tb_cis_op_visiting_record_202001_202012.csv']
                 
file_name_list = ['tb_cis_op_visiting_record_201401_201806.csv',
                  'tb_cis_op_visiting_record_201807_201912.csv',\
                  'tb_cis_op_visiting_record_202001_202012.csv',\
                  'tb_cis_op_visiting_record_202101_202112.csv',\
                  'tb_cis_op_visiting_record_202201_202205.csv']

result=[]
t_start=time()
for file_name in file_name_list:
    print(file_name)
    B2_df = spark.read.option("inferSchema","true").option("delimiter", ",").option("header","true").csv(os.path.join(path1, file_name))
    B2_df.createOrReplaceTempView("B2_00")
    start=time()
    
    print('读入数据耗时：%.2f秒'%(time()-start))
    df = spark.sql("""
    select
    count(*)as TOTAL_NUM
    ,count(VISITING_DATE)as VISITING_DATE_NOTNULL
    ,count(ID_NUMBER)as ID_NUMBER_NOTNULL
    ,count(AGE)as AGE_NOTNULL
    ,count(GENDER)as GENDER_NOTNULL
    ,count(DISEASE_DIAGNOSIS_CODE)as DISEASE_DIAGNOSIS_CODE_NOTNULL
    ,count(NAME_OF_DISEASE_DIAGNOSIS)as NAME_OF_DISEASE_DIAGNOSIS_NOTNULL
    ,count(DIAGNOSIS_DESCRIPTION)as DIAGNOSIS_DESCRIPTION_NOTNULL
    ,count(VISITING_TYPE_CODE)as VISITING_TYPE_CODE_NOTNULL
    ,count(VISITING_TYPE_NAME)as VISITING_TYPE_NAME_NOTNULL
    ,count(ICD_CODE)as ICD_CODE_NOTNULL
    ,count(area)as AREA_NOTNULL
    ,sum(case when MEDICAL_INSTITUTION_CODE is not null and FILE_NUMBER is not null then 1 else 0 end) as flag1
    ,sum(case when CARD_NUMBER is not null and CARD_TYPE is not null then 1 else 0 end) as flag2
    ,sum(case when GENDER is null and id_gender is null then 0 else 1 end) as flag3
    ,sum(case when VISITING_DATE is null and DATE_OF_DIAGNOSIS is null then 0 else 1 end) as flag4
    ,sum(case when BIRTH_DATE is null and id_birth_date is null then 0 else 1 end) as flag5
    ,sum(case when DISEASE_DIAGNOSIS_CODE is null and NAME_OF_DISEASE_DIAGNOSIS is null then 0 else 1 end) as flag6
    ,sum(case when ICD_CODE is not null and area is not null then 1 else 0 end) as flag7
    from B2_00""").toPandas()

    for col in df.columns:
        if (col != 'TOTAL_NUM'):
            df.insert(df.columns.get_loc(col)+1,col+'_RATE' ,df[col]/df['TOTAL_NUM']*100)
            
    result.append(df)

    print('处理数据耗时：%.2f秒'%(time()-start))
result_df =pd.concat(result)
result_df.insert(0,'File',file_name_list)
result_df.columns=['文件名','数据总量','VISITING_DATE非空数据量','VISITING_DATE完整率','ID_NUMBER非空数据量','ID_NUMBER完整率',\
                   'AGE非空数据量','AGE完整率','GENDER非空数据量','GENDER完整率',\
                   'DISEASE_DIAGNOSIS_CODE非空数据量','DISEASE_DIAGNOSIS_CODE完整率','NAME_OF_DISEASE_DIAGNOSIS非空数据量',\
                   'NAME_OF_DISEASE_DIAGNOSIS完整率','DIAGNOSIS_DESCRIPTION非空数据量','DIAGNOSIS_DESCRIPTION完整率',\
                   'VISITING_TYPE_CODE非空数据量','VISITING_TYPE_CODE完整率','VISITING_TYPE_NAME非空数据量',\
                   'VISITING_TYPE_NAME完整率','ICD_CODE非空数据量','ICD_CODE完整率',\
                   'AREA非空数据量','AREA完整率',\
                   'MEDICAL_INSTITUTION_CODE,FILE_NUMBER字段组合均非空数据量','MEDICAL_INSTITUTION_CODE,FILE_NUMBER字段组合完整率',\
                   'CARD_TYPE,CARD_NUMBER字段组合均非空数据量','CARD_TYPE,CARD_NUMBER字段组合完整率',\
                   'GENDER,id_gender至少一个非空数据量','GENDER,id_gender至少一个非空完整率',\
                   'VISITING_DATE,DATE_OF_DIAGNOSIS至少一个非空数据量','VISITING_DATE,DATE_OF_DIAGNOSIS至少一个非空完整率',\
                   'BIRTH_DATE,id_birth_date至少一个非空数据量','BIRTH_DATE,id_birth_date至少一个非空完整率',\
                   'DISEASE_DIAGNOSIS_CODE,NAME_OF_DISEASE_DIAGNOSIS至少一个非空数据量',\
                   'DISEASE_DIAGNOSIS_CODE,NAME_OF_DISEASE_DIAGNOSIS至少一个非空完整率',\
                 'ICD_CODE,CITY均非空数据量','ICD_CODE,CITY字段组合完整率']
result_df.to_excel(r'D:/Output/chenbingying/B1统计/B1关键字段数据质量统计v01.xls',index = False)
print('总数据耗时：%.2f秒'%(time()-t_start))

tb_cis_op_visiting_record_201401_201806.csv
读入数据耗时：0.00秒
处理数据耗时：94.99秒
tb_cis_op_visiting_record_201807_201912.csv
读入数据耗时：0.00秒
处理数据耗时：69.25秒
tb_cis_op_visiting_record_202001_202012.csv
读入数据耗时：0.00秒
处理数据耗时：78.42秒
tb_cis_op_visiting_record_202101_202112.csv
读入数据耗时：0.00秒
处理数据耗时：96.96秒
tb_cis_op_visiting_record_202201_202205.csv
读入数据耗时：0.00秒
处理数据耗时：57.26秒
总数据耗时：836.81秒


C:\Users\User2\AppData\Local\Temp/ipykernel_20236/1935839397.py:67: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  result_df.to_excel(r'D:/Output/chenbingying/B1统计/B1关键字段数据质量统计v01.xls',index = False)


In [ ]:
result_df